# Dataset and configuration

In this part, we will introduce how to prepare a new dataset and its configuration file, and the basic usage of `UserConfig` and `DataModule`. You will be able to run benchmarks on your own dataset after reading this part.

## The dataset

We provide a randomly generated sample dataset (`data/sample.csv`) and its configuration file (`configs/sample.py`) in the repository. First, let's check the content of `sample.csv`. It contains 256 data points, 10 continuous features (namely `cont_0` to `cont_9`), 10 categorical features (namely `cat_0` to `cat_9`), and one target column `target`.

**Remark**: The dataset file should not contain an index column.

**Remark**: Both `.csv` and `.xlsx` are supported. We recommend `.csv` files for their efficiency.

In [1]:
import pandas as pd

prefix = "../../../../"
pd.read_csv(prefix + "data/sample.csv")

,cont_0,cont_1,cont_2,cont_3,cont_4,cont_5,cont_6,cont_7,cont_8,cont_9,...,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,target,target_binary,target_multi_class
0,-1.306527,NaN,-0.118164,-0.159573,1.658131,-1.346718,-0.680178,-1.334258,0.666383,-0.460720,...,0,2,category_4,3,4,4,3,-71.084217,0,1
1,2.011257,NaN,0.195070,0.527004,-0.044595,0.616887,-1.781563,0.354758,-0.729045,0.196557,...,4,3,category_3,3,1,3,2,13.415675,1,2
2,-1.216077,NaN,-0.743672,0.730184,0.140672,1.272954,-0.159012,-0.475175,0.240057,0.100159,...,0,4,category_3,4,1,0,2,-47.492280,0,2
3,0.559299,NaN,-0.431096,-0.809627,-1.063696,-0.860153,0.572751,-0.467441,0.677557,1.307184,...,4,1,category_3,4,2,0,0,-94.482614,1,2
4,0.910179,NaN,0.786328,-0.042257,0.317218,0.379152,-0.466419,-0.017020,-0.944446,-0.410050,...,1,0,category_2,0,2,3,0,195.819531,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0.280442,-0.206904,0.841631,0.880179,-0.993124,-1.570623,-0.249459,0.643314,0.049495,0.493837,...,1,2,category_2,2,3,0,2,-171.249549,0,0
252,-1.165150,-1.070753,0.465662,1.054452,0.900826,-0.179925,-1.536244,1.178780,1.488252,1.895889,...,4,2,category_4,4,2,1,1,23.708442,0,2
253,-0.069856,-0.186691,-1.021913,-1.143641,0.250114,1.040239,-1.150438,0.258798,-0.836111,0.642211,...,0,3,category_3,2,2,2,2,-33.414215,1,1
254,-1.031482,-0.860262,-0.061638,0.328301,-1.429991,-1.048170,-1.432735,0.607112,0.087531,0.938747,...,0,0,category_3,4,1,4,4,-359.199191,0,4


## The configuration file

A configuration file contains a dictionary stating modified values compared to a given default configuration.

**Remark**: The dataset file can be a `.py` file containing a `dict` object named `cfg`, or a `.json` file.

### The default configuration

To see the default values, use `tabensemb.config.UserConfig`, which inherits `dict`.

In [2]:
from tabensemb.config import UserConfig
from tabensemb.utils import pretty
import tabensemb

tabensemb.setting["default_config_path"] = prefix + "configs"

cfg = UserConfig("sample")
print(pretty(cfg.defaults()))

{
	'database': 'sample',
	'task': None,
	'loss': None,
	'bayes_opt': False,
	'bayes_calls': 50,
	'bayes_epoch': 30,
	'patience': 100,
	'epoch': 300,
	'lr': 0.001,
	'weight_decay': 1e-09,
	'batch_size': 1024,
	'layers': [
		64,
		128,
		256,
		128,
		64
	],
	'SPACEs': {
		'lr': {
			'type': 'Real',
			'low': 0.0001,
			'high': 0.05,
			'prior': 'log-uniform'
		},
		'weight_decay': {
			'type': 'Real',
			'low': 1e-09,
			'high': 0.05,
			'prior': 'log-uniform'
		},
		'batch_size': {
			'type': 'Categorical',
			'categories': [
				64,
				128,
				256,
				512,
				1024,
				2048
			]
		}
	},
	'data_splitter': 'RandomSplitter',
	'split_ratio': [
		0.6,
		0.2,
		0.2
	],
	'data_imputer': 'MissForestImputer',
	'data_processors': [
		(
			'CategoricalOrdinalEncoder',
			{
			}
		),
		(
			'NaNFeatureRemover',
			{
			}
		),
		(
			'VarianceFeatureSelector',
			{
				'thres': 1
			}
		),
		(
			'StandardScaler',
			{
			}
		)
	],
	'data_derivers': [
	],
	'feature_names_type': {
	},
	'categor

### The configuration of the given sample dataset

`configs/sample.py` contains the following contents:
```python
cfg = {
    "database": "sample",
    "feature_types": ["Continuous", "Categorical", "Derived"],
    "feature_names_type": {
        "cont_0": 0,
        "cont_1": 0,
        "cont_2": 0,
        "cont_3": 0,
        "cont_4": 0,
        "cat_0": 1,
        "cat_1": 1,
        "cat_2": 1,
    },
    "categorical_feature_names": [
        "cat_0",
        "cat_1",
        "cat_2",
    ],
    "label_name": ["target"],
}
```
Load `configs/sample.py` and see the changes.

In [3]:
cfg = UserConfig("sample")
print(pretty(cfg))

{
	'database': 'sample',
	'task': None,
	'loss': None,
	'bayes_opt': False,
	'bayes_calls': 50,
	'bayes_epoch': 30,
	'patience': 100,
	'epoch': 300,
	'lr': 0.001,
	'weight_decay': 1e-09,
	'batch_size': 1024,
	'layers': [
		64,
		128,
		256,
		128,
		64
	],
	'SPACEs': {
		'lr': {
			'type': 'Real',
			'low': 0.0001,
			'high': 0.05,
			'prior': 'log-uniform'
		},
		'weight_decay': {
			'type': 'Real',
			'low': 1e-09,
			'high': 0.05,
			'prior': 'log-uniform'
		},
		'batch_size': {
			'type': 'Categorical',
			'categories': [
				64,
				128,
				256,
				512,
				1024,
				2048
			]
		}
	},
	'data_splitter': 'RandomSplitter',
	'split_ratio': [
		0.6,
		0.2,
		0.2
	],
	'data_imputer': 'MissForestImputer',
	'data_processors': [
		(
			'CategoricalOrdinalEncoder',
			{
			}
		),
		(
			'NaNFeatureRemover',
			{
			}
		),
		(
			'VarianceFeatureSelector',
			{
				'thres': 1
			}
		),
		(
			'StandardScaler',
			{
			}
		)
	],
	'data_derivers': [
	],
	'feature_names_type': {
		'cont_0': 0

### Descriptions of keys in a configuration file

* `database`: The name of the database file. The file should be placed in the script directory or in `tabensemb.setting["default_data_path"]`. If no postfix (`.csv` or `.xlsx`) is provided, the program automatically searches for a matched postfix. If both `.csv` and `.xlsx` exist, an exception will be raised.
* `task`: "regression" for regression tasks, "binary" for binary classifications, and "multiclass" for multiclass classifications. If left None, the task will be guessed from the type of the target. If the target is of the type `object` or integers, "binary" or "multiclass" is guessed depending on the number of unique targets; otherwise, "regression" is guessed.
* `loss`: "mse" (default) or "mae" for regression tasks, and "cross_entropy" for classification tasks. This loss will be used across all model bases. If left None, "mse" or "cross_entropy" will be used.
* `bayes_opt`: Perform gaussian-process-based Bayesian hyperparameter optimization (HPO) using the `scikit-optimize` package when training each model.
* `bayes_calls`: The number of calls of the Bayesian HPO. During each call, the model will be trained given a set of hyperparameters, and then the metric on the validation set will be returned to the Bayesian HPO process.
* `bayes_epoch`: The number of epochs during each Bayesian HPO call.
* `patience`: Early stopping patience. If the metric on the validation set does not improve after `patience` epochs, the training process terminates and the best model is loaded.
* `epoch`: Total epochs to train each model.
* `lr`: Initial learning rate.
* `weight_decay`: Initial weight_decay (for a `torch.optim.Adam` optimizer)
* `batch_size`: Initial batch_size.
* `layers`: Default hidden layers for some models.
* `SPACEs`: Default bayesian HPO spaces for `lr`, `weight_decay`, and `batch_size`. The key `type` determines the `skopt.space`, and the rest of the keys determines its arguments.
* `data_splitter`: The dataset splitting method to split training/validation/testing sets. See `tabensemb.data.datasplitter.splitter_mapping` for available classes.
* `split_ratio`: The ratio of training/validation/testing sets.
* `data_imputer`: The imputation method for `NaN` values. See `tabensemb.data.dataimputer.imputer_mapping` for available classes.
* `data_processors`: A list of data processing steps and their corresponding arguments. See `tabensemb.data.dataprocessor.processor_mapping` for available classes. See API docs for definitions of arguments.
* `data_derivers`: A list of feature augmentation steps and their corresponding arguments. Some fix arguments are
    * `stacked`: `True` to append the derived feature to continuous features and the final `DataFrame` representing the processed dataset. `False` to leave it as an unstacked feature (mostly for multi-modal data)
    * `intermediate`: `True` to ignore the derived feature in continuous features even when `stacked=True`, but still append the feature to the `DataFrame`.

    See `tabensemb.data.dataderiver.deriver_mapping` for available classes. See API docs or `_required_cols` of each class for its additional arguments.
* `feature_types`: General types of features. `Categorical` and `Derived` are necessary for training and plotting.
* `feature_names_type`: A dictionary stating used features and their types. In this example, for continuous features the value is 0, which is the index of `Continuous` in `feature_types`, and for categorical features, the value is 1 which is the index of `Categorical` in `feature_types`.
* `categorical_feature_names`: Just repeat features that are `Categorical` here.
* `label_name`: The predicted target.

## Use the configuration file to load the dataset

The `DataModule` requires a `UserConfig` to load the dataset, then initialize and run all data processing steps on the dataset or an upcoming new dataset. The following lines load the dataset and present the loaded and processed `DataFrame` without imputation.

In [4]:
from tabensemb.data import DataModule
tabensemb.setting["default_data_path"] = prefix + "data"

datamodule = DataModule(cfg)
datamodule.load_data()
datamodule.get_not_imputed_df()

Dataset size: 153 51 52


,cont_0,cont_1,cont_2,cont_3,cont_4,cont_5,cont_6,cont_7,cont_8,cont_9,...,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,target,target_binary,target_multi_class
0,-1.306527,NaN,-0.118164,-0.159573,1.658131,-1.346718,-0.680178,-1.334258,0.666383,-0.460720,...,0,2,category_4,3,4,4,3,-71.084217,0,1
1,2.011257,NaN,0.195070,0.527004,-0.044595,0.616887,-1.781563,0.354758,-0.729045,0.196557,...,4,3,category_3,3,1,3,2,13.415675,1,2
2,-1.216077,NaN,-0.743672,0.730184,0.140672,1.272954,-0.159012,-0.475175,0.240057,0.100159,...,0,4,category_3,4,1,0,2,-47.492280,0,2
3,0.559299,NaN,-0.431096,-0.809627,-1.063696,-0.860153,0.572751,-0.467441,0.677557,1.307184,...,4,1,category_3,4,2,0,0,-94.482614,1,2
4,0.910179,NaN,0.786328,-0.042257,0.317218,0.379152,-0.466419,-0.017020,-0.944446,-0.410050,...,1,0,category_2,0,2,3,0,195.819531,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0.280442,-0.206904,0.841631,0.880179,-0.993124,-1.570623,-0.249459,0.643314,0.049495,0.493837,...,1,2,category_2,2,3,0,2,-171.249549,0,0
252,-1.165150,-1.070753,0.465662,1.054452,0.900827,-0.179925,-1.536244,1.178780,1.488252,1.895889,...,4,2,category_4,4,2,1,1,23.708442,0,2
253,-0.069856,-0.186691,-1.021913,-1.143641,0.250114,1.040239,-1.150438,0.258798,-0.836111,0.642211,...,0,3,category_3,2,2,2,2,-33.414215,1,1
254,-1.031482,-0.860262,-0.061638,0.328301,-1.429991,-1.048170,-1.432735,0.607112,0.087531,0.938747,...,0,0,category_3,4,1,4,4,-359.199191,0,4


`DataModule.df` present the imputed `DataFrame`.

In [5]:
datamodule.df

,cont_0,cont_1,cont_2,cont_3,cont_4,cont_5,cont_6,cont_7,cont_8,cont_9,...,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,target,target_binary,target_multi_class
0,-1.306527,0.851431,-0.118164,-0.159573,1.658131,-1.346718,-0.680178,-1.334258,0.666383,-0.460720,...,0,2,category_4,3,4,4,3,-71.084217,0,1
1,2.011257,0.205221,0.195070,0.527004,-0.044595,0.616887,-1.781563,0.354758,-0.729045,0.196557,...,4,3,category_3,3,1,3,2,13.415675,1,2
2,-1.216077,0.851431,-0.743672,0.730184,0.140672,1.272954,-0.159012,-0.475175,0.240057,0.100159,...,0,4,category_3,4,1,0,2,-47.492280,0,2
3,0.559299,-0.487775,-0.431096,-0.809627,-1.063696,-0.860153,0.572751,-0.467441,0.677557,1.307184,...,4,1,category_3,4,2,0,0,-94.482614,1,2
4,0.910179,0.205221,0.786328,-0.042257,0.317218,0.379152,-0.466419,-0.017020,-0.944446,-0.410050,...,1,0,category_2,0,2,3,0,195.819531,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,0.280442,-0.206904,0.841631,0.880179,-0.993124,-1.570623,-0.249459,0.643314,0.049495,0.493837,...,1,2,category_2,2,3,0,2,-171.249549,0,0
252,-1.165150,-1.070753,0.465662,1.054452,0.900827,-0.179925,-1.536244,1.178780,1.488252,1.895889,...,4,2,category_4,4,2,1,1,23.708442,0,2
253,-0.069856,-0.186691,-1.021913,-1.143641,0.250114,1.040239,-1.150438,0.258798,-0.836111,0.642211,...,0,3,category_3,2,2,2,2,-33.414215,1,1
254,-1.031482,-0.860262,-0.061638,0.328301,-1.429991,-1.048170,-1.432735,0.607112,0.087531,0.938747,...,0,0,category_3,4,1,4,4,-359.199191,0,4


`DataModule.train_indices`, `DataModule.val_indices`, and `DataModule.test_indices` represent indices of training/validation/testing sets, respectively.

In [6]:
datamodule.train_indices, datamodule.val_indices, datamodule.test_indices

(array([ 63, 185, 141,  48,  54, 219, 232, 108,  10, 190,  32, 228,  24,
        230, 161,   0,   7, 173, 144, 168,  31,  43, 115, 133,  83,  86,
         61, 127, 105,   8,  42,  77, 217,  90, 170, 123, 187,  88,  97,
        171, 102, 223,  25, 218,  56, 242, 184, 208,  17, 221, 201,  19,
        188, 118, 165, 247, 153, 101,  39, 207, 139,   9,  26, 229,  66,
        238,   3, 250,  22, 197,  95,  38, 107,  33, 134, 166,  76, 215,
         73, 225, 149, 182,  49,  75, 234,  74, 240, 111,  99,  89, 206,
         55,  82,  68, 237, 129, 214,   6,  28, 192, 193, 236,  37,  53,
        154,  20,  47, 180,  98,   1, 128,  11, 224, 222, 147, 255, 174,
         16,  60,  45,  87,  12, 233, 204, 210, 162,  67, 156, 169,  65,
        176, 125,  27,  34, 251, 227, 113,  72, 202,  15, 241,  92, 252,
        110, 211, 112,  80,  71, 116, 130, 205,  59, 198]),
 array([135,  46, 119, 235, 199, 122, 183,  51,  78, 249, 212, 120, 104,
         30, 172, 131,  91, 159,  57,   2, 216, 196, 152, 158,  

For detailed functionalities of `DataModule`, please check the API documentation.

## A `Trainer` does all things for you

Indeed, a user does not need to manually generate a `UserConfig` or a `DataModule` because `Trainer` does all the above steps. After calling `Trainer.load_config` and `Trainer.load_data`, a `UserConfig` instance containing configurations, a `DataModule` instance containing processing steps, and loaded data are generated and can be accessed by `Trainer.args` and `Trainer.datamodule`, respectively.


In [7]:
from tabensemb.trainer import Trainer

tabensemb.setting["default_output_path"] = prefix + "output"
trainer = Trainer(device="cpu")
trainer.load_config("sample")
trainer.load_data()
type(trainer.args), type(trainer.datamodule)

The project will be saved to ../../../../output/sample/2023-09-06-20-19-14-0_sample
Dataset size: 153 51 52
Data saved to ../../../../output/sample/2023-09-06-20-19-14-0_sample (data.csv and tabular_data.csv).


(tabensemb.config.user_config.UserConfig, tabensemb.data.datamodule.DataModule)